# Paper link and github
Paper : https://arxiv.org/pdf/2004.10934.pdf 

GitHub : https://github.com/pythonlessons/TensorFlow-2.x-YOLOv3

In [1]:
from google.colab import drive
drive.mount('/content/gdrive')
%cd /content/gdrive/My Drive/CV projects/streetViewHouseNumbers/TensorFlow-2.x-YOLOv3-master/
!ls

Mounted at /content/gdrive
/content/gdrive/My Drive/CV projects/streetViewHouseNumbers/TensorFlow-2.x-YOLOv3-master
checkpoints		  IMAGES	     requirements.txt
Collect_training_data.py  LICENSE	     streetView
custom_dataset		  log		     streetViewHouseNumbersYolo4.ipynb
deep_sort		  mnist		     tools
detection_custom.py	  model_data	     train.py
detection_demo.py	  object_tracker.py  wandb
detect_mnist.py		  __pycache__	     yolov3
evaluate_mAP.py		  README.md	     YOLOv3_colab_training.ipynb


In [2]:
!nvidia-smi

Sat Nov 27 19:06:03 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 495.44       Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla V100-SXM2...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   42C    P0    26W / 300W |      0MiB / 16160MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [2]:
import os
import cv2
import json
import random
from tqdm import tqdm
import numpy as np
import pandas as pd 
import matplotlib
import matplotlib.pyplot as plt
%matplotlib inline  
import tensorflow as tf
from yolov3.yolov4 import Create_Yolo
from yolov3.utils import * 
from yolov3.configs import *

In [ ]:
# pip install -r ./requirements.txt
%cd /content/gdrive/My Drive/CV projects/streetViewHouseNumbers/TensorFlow-2.x-YOLOv3-master/
# yolov3
!wget -P model_data https://pjreddie.com/media/files/yolov3.weights
# yolov3-tiny
!wget -P model_data https://pjreddie.com/media/files/yolov3-tiny.weights
# yolov4
!wget -P model_data https://github.com/AlexeyAB/darknet/releases/download/darknet_yolo_v3_optimal/yolov4.weights
# yolov4-tiny
!wget -P model_data https://github.com/AlexeyAB/darknet/releases/download/darknet_yolo_v4_pre/yolov4-tiny.weights

--2021-11-22 05:29:49--  https://pjreddie.com/media/files/yolov3.weights
Resolving pjreddie.com (pjreddie.com)... 128.208.4.108
Connecting to pjreddie.com (pjreddie.com)|128.208.4.108|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 248007048 (237M) [application/octet-stream]
Saving to: ‘model_data/yolov3.weights.6’

yolov3.weights.6    100%[===================>] 236.52M  35.3MB/s    in 7.0s    

2021-11-22 05:29:56 (33.7 MB/s) - ‘model_data/yolov3.weights.6’ saved [248007048/248007048]

--2021-11-22 05:29:57--  https://pjreddie.com/media/files/yolov3-tiny.weights
Resolving pjreddie.com (pjreddie.com)... 128.208.4.108
Connecting to pjreddie.com (pjreddie.com)|128.208.4.108|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 35434956 (34M) [application/octet-stream]
Saving to: ‘model_data/yolov3-tiny.weights’

yolov3-tiny.weights 100%[===================>]  33.79M  27.4MB/s    in 1.2s    

2021-11-22 05:29:58 (27.4 MB/s) - ‘model_data/yolov

# Download and format the data

In [3]:
data_links = pd.read_csv('/content/gdrive/My Drive/CV projects/streetViewHouseNumbers/data/mchar_data_list_0515.csv') 
data_links

,file,size,link
0,mchar_train.zip,345.91MB,http://tianchi-competition.oss-cn-hangzhou.ali...
1,mchar_train.json,3.16MB,http://tianchi-competition.oss-cn-hangzhou.ali...
2,mchar_val.zip,200.16MB,http://tianchi-competition.oss-cn-hangzhou.ali...
3,mchar_val.json,1.03MB,http://tianchi-competition.oss-cn-hangzhou.ali...
4,mchar_test_a.zip,370.6MB,http://tianchi-competition.oss-cn-hangzhou.ali...
5,mchar_sample_submit_A.csv,507.83KB,http://tianchi-competition.oss-cn-hangzhou.ali...


In [4]:
for i in range(4, 6): 
  files_path = tf.keras.utils.get_file(data_links["file"].iloc[i], data_links["link"].iloc[i], extract=True) 
print("files_path: ", files_path) 

532480/520020 [==============================] - 1s 3us/step
files_path:  /root/.keras/datasets/mchar_sample_submit_A.csv


In [6]:
%cd /root/.keras/datasets/
!ls

/root/.keras/datasets
__MACOSX  mchar_sample_submit_A.csv  mchar_test_a  mchar_test_a.zip


In [39]:
def make_date(images_path, labels_path, file): 
    sorted_img_name = sorted(os.listdir(images_path)) 
    sorted_img_path = [images_path + '/' + img for img in sorted_img_name] 
    json_file = open(labels_path, 'r')
    label_data = json.load(json_file)
    json_file.close()

    with open(file, "w") as wf:
        for i in range(len(sorted_img_name)): 
            image_path = sorted_img_path[i] 
            label_info = label_data[sorted_img_name[i]]
            labels = label_info['label']
            annotation = image_path
            for j in range(len(labels)): 
                xmin = str(int(1 if label_info['left'][j] < 0 else label_info['left'][j])) 
                ymin = str(int(1 if label_info['top'][j] < 0 else label_info['top'][j])) 
                xmax = str(int(1 if label_info['left'][j] + label_info['width'][j] < 0 else label_info['left'][j] + label_info['width'][j])) 
                ymax = str(int(1 if label_info['top'][j] + label_info['height'][j] < 0 else label_info['top'][j] + label_info['height'][j])) 
                annotation += ' ' + ','.join([xmin, ymin, xmax, ymax, str(labels[j])])
            # print("=> %s" %annotation)
            wf.write(annotation + "\n")

In [ ]:
# make train data
images_path = "/root/.keras/datasets/mchar_train"
labels_path = "/root/.keras/datasets/mchar_train.json"
file = "/content/gdrive/My Drive/CV projects/streetViewHouseNumbers/TensorFlow-2.x-YOLOv3-master/streetView/streetView_train.txt"
make_date(images_path, labels_path, file)

In [ ]:
# make validation data
images_path = "/root/.keras/datasets/mchar_val"
labels_path = "/root/.keras/datasets/mchar_val.json"
file = "/content/gdrive/My Drive/CV projects/streetViewHouseNumbers/TensorFlow-2.x-YOLOv3-master/streetView/streetView_val.txt"
make_date(images_path, labels_path, file)

# Train Yolo4 model

In [7]:
%cd /content/gdrive/My Drive/CV projects/streetViewHouseNumbers/TensorFlow-2.x-YOLOv3-master/
!python train.py 

/content/gdrive/My Drive/CV projects/streetViewHouseNumbers/TensorFlow-2.x-YOLOv3-master


In [7]:
def sort_func(x): 
    return x[0]

def detect_image_class(Yolo, image_path, input_size=416, show=False, CLASSES=YOLO_COCO_CLASSES, score_threshold=0.3, iou_threshold=0.45):
    original_image      = cv2.imread(image_path)
    original_image      = cv2.cvtColor(original_image, cv2.COLOR_BGR2RGB)
    original_image      = cv2.cvtColor(original_image, cv2.COLOR_BGR2RGB)

    image_data = image_preprocess(np.copy(original_image), [input_size, input_size])
    image_data = image_data[np.newaxis, ...].astype(np.float32)

    if YOLO_FRAMEWORK == "tf":
        pred_bbox = Yolo.predict(image_data)
        
    pred_bbox = [tf.reshape(x, (-1, tf.shape(x)[-1])) for x in pred_bbox]
    pred_bbox = tf.concat(pred_bbox, axis=0)
    
    bboxes = postprocess_boxes(pred_bbox, original_image, input_size, score_threshold)
    bboxes = nms(bboxes, iou_threshold, method='nms')

    bboxes.sort(key=sort_func)

    image_class = [int(x[5]) for x in bboxes]

    return image_class

# Check performance in validation set

In [8]:
def make_pred(TRAIN_MODEL_NAME, label_txt, total_num): 
    predictions = {} 
    for ID in tqdm(range(total_num)): 
        image_info = open(label_txt).readlines()[ID].split()

        image_path = image_info[0]
        image = os.path.basename(image_path)

        yolo = Create_Yolo(input_size=YOLO_INPUT_SIZE, CLASSES=TRAIN_CLASSES)
        yolo.load_weights(f"./checkpoints/{TRAIN_MODEL_NAME}") # use keras weights

        prediction = detect_image_class(yolo, image_path, input_size=YOLO_INPUT_SIZE, show=False, CLASSES=TRAIN_CLASSES)
        predictions[image] = prediction 

    return predictions


In [ ]:
# predict validation set 
TRAIN_MODEL_NAME = 'yolov4_custom_Tiny'
val_predictions = {}
label_txt = "/content/gdrive/My Drive/CV projects/streetViewHouseNumbers/TensorFlow-2.x-YOLOv3-master/streetView/streetView_val.txt"
num_val = len(open(label_txt).readlines())

%cd /content/gdrive/My Drive/CV projects/streetViewHouseNumbers/TensorFlow-2.x-YOLOv3-master/
val_predictions = make_pred(TRAIN_MODEL_NAME, label_txt, num_val)

In [13]:
def compare(y, pred): 
    if len(y) != len(pred): 
        return False 
    for i in range(len(y)): 
        if y[i] != pred[i]: 
            return False 
    return True 

# calculate accuracy
correct = 0 
sorted_img_name = sorted(os.listdir("/root/.keras/datasets/mchar_val")) 
label_data = json.load(open("/root/.keras/datasets/mchar_val.json", 'r')) 
for i in range(len(sorted_img_name)): 
    image = sorted_img_name[i] 
    labels = label_data[image]['label']
    prediction = val_predictions[image]
    if compare(labels, prediction) == True: 
        correct += 1 

In [21]:
acc = correct / len(sorted_img_name)
acc

0.6303

# Predict testing data

In [5]:
#make testing data
def make_date(images_path, file): 
    sorted_img_name = sorted(os.listdir(images_path)) 
    sorted_img_path = [images_path + '/' + img for img in sorted_img_name] 
    with open(file, "w") as wf:
        for i in range(len(sorted_img_name)): 
            wf.write(sorted_img_path[i] + "\n")

images_path = "/root/.keras/datasets/mchar_test_a"
file = "/content/gdrive/My Drive/CV projects/streetViewHouseNumbers/TensorFlow-2.x-YOLOv3-master/streetView/streetView_test.txt"
make_date(images_path, file)

In [6]:
TRAIN_MODEL_NAME = 'yolov4_custom_Tiny'
test_predictions = {}
label_txt = "/content/gdrive/My Drive/CV projects/streetViewHouseNumbers/TensorFlow-2.x-YOLOv3-master/streetView/streetView_test.txt"
num_test = len(open(label_txt).readlines())
print("number of testing samples: ", num_test)

# %cd /content/gdrive/My Drive/CV projects/streetViewHouseNumbers/TensorFlow-2.x-YOLOv3-master/
# test_predictions = make_pred(TRAIN_MODEL_NAME, label_txt, num_val)

number of testing samples:  40000


In [7]:
def write_to_file(mapping): 
    upload_data = [] 
    for image in mapping: 
        numbers = mapping[image]
        numbers = [str(x) for x in numbers]
        plate = ''.join(numbers)
        upload_data.append((image, plate))
    upload_df = pd.DataFrame(upload_data, columns=['file_name', 'file_code'])
    upload_df.to_csv(path_or_buf = '/content/gdrive/My Drive/CV projects/streetViewHouseNumbers/outputs/yolo4.csv', index = False)

In [ ]:
%cd /content/gdrive/My Drive/CV projects/streetViewHouseNumbers/TensorFlow-2.x-YOLOv3-master/
test_predictions = make_pred(TRAIN_MODEL_NAME, label_txt, num_test)
write_to_file(test_predictions)

/content/gdrive/My Drive/CV projects/streetViewHouseNumbers/TensorFlow-2.x-YOLOv3-master


 28%|██▊       | 5658/20000 [2:18:29<6:01:29,  1.51s/it]

In [17]:
len(test_predictions)

10

In [18]:
test_predictions

{'020000.png': [6, 6, 9],
 '020001.png': [1, 8],
 '020002.png': [8, 0],
 '020003.png': [6, 7, 2],
 '020004.png': [5, 9],
 '020005.png': [2, 0],
 '020006.png': [1, 3, 2, 2],
 '020007.png': [6, 1, 3],
 '020008.png': [2, 0, 8],
 '020009.png': [5, 1, 9]}